<a href="https://colab.research.google.com/github/IlyaDenisov88/dataenj/blob/main/PySpark/Spark_submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Spark-submit

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=bbfda2ae5300c9e768ccabf880a0cb89bfa053aab1d3c28bcc9ab9e833f14d84
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName("Tungsten Example") \
    .config("spark.sql.codegen.wholeStage", "true") \
    .getOrCreate()

data = [(i, f"Name_{i % 5}", i % 3, i % 2) for i in range(1, 1000001)]
df = spark.createDataFrame(data, ["id", "name", "mod3", "mod2"])

# Применяем фильтр и агрегацию, чтобы увидеть работу Tungsten
result = df.filter(col("mod2") == 1) \
           .groupBy("mod3") \
           .agg({"id": "sum"}) \
           .orderBy("mod3")

In [5]:
!spark-submit --master local\[4\] \
             --name "Tungsten Example" \
             --conf spark.sql.codegen.wholeStage=true \
             --conf spark.executor.memory=2g \
             main.py# здесь сохранен текст пред ячейки

Error: Missing application resource.

Usage: spark-submit [options] <app jar | python file | R file> [app arguments]
Usage: spark-submit --kill [submission ID] --master [spark://...]
Usage: spark-submit --status [submission ID] --master [spark://...]
Usage: spark-submit run-example [options] example-class [example args]

Options:
  --master MASTER_URL         spark://host:port, mesos://host:port, yarn,
                              k8s://https://host:port, or local (Default: local[*]).
  --deploy-mode DEPLOY_MODE   Whether to launch the driver program locally ("client") or
                              on one of the worker machines inside the cluster ("cluster")
                              (Default: client).
  --class CLASS_NAME          Your application's main class (for Java / Scala apps).
  --name NAME                 A name of your application.
  --jars JARS                 Comma-separated list of jars to include on the driver
                              and executor classpaths